# Project Corona
## Philosophy and History of Science with Computational Means
Prof Dr. Gerd Graßhoff
### Filter dataframe, NLTP

# Libraries, data

In [2]:
import pandas as pd
import spacy
from spacy import displacy
from spacy.matcher import Matcher
import re
from collections import Counter

In [3]:
# Import English library
nlp = spacy.load('en_core_web_sm')

In [4]:
dfRem=pd.read_json("dfRemdesivierResearchObjects.json")[["title","abstract"]]

In [5]:
len(dfRem)

2180

In [6]:
dfRem.head(2)

,title,abstract
0,Evaluation of the efficacy and safety of intra...,BACKGROUND: Coronavirus disease 2019 (COVID-19...
1,Role of adjunctive treatment strategies in COV...,The coronavirus disease (COVID-19) pandemic ha...


In [7]:
dfRem=dfRem.dropna(subset=["abstract"])
len(dfRem)

2135

# Similarity

In [8]:
suche="We show"
def filter(zeile):
    cond=zeile["abstract"].str.contains(suche,na=False)
    return(cond)

In [9]:
df=dfRem[filter]
df

,title,abstract
341,In silico detection of SARS-CoV-2 specific B-c...,Abstract Rapid generation of diagnostics is pa...
537,Structure based drug discovery by virtual scre...,&lt;p&gt;Background&lt;/p&gt;&lt;p&gt;The curr...
649,Defining the Pandemic at the State Level: Sequ...,"In December of 2019, a novel coronavirus, SARS..."
685,COVID-19 research in Wikipedia,Wikipedia is one of the main sources of free k...
1542,SARS-CoV-2 and SARS-CoV differ in their cell t...,SARS-CoV-2 is a novel coronavirus currently ca...


# Pattern matching
https://spacy.io/usage/rule-based-matching

https://explosion.ai/demos/matcher?

In [10]:
matcher = Matcher(nlp.vocab)

In [11]:
pattern = [{"LOWER":"we"},{'POS': 'VERB'}]
matcher.add("matching",None,pattern)

In [13]:
listverb=[]
def actionverb(a):
    abstract=nlp(a)
    matches=matcher(abstract)
    for _,start,end in matches:
        verb=abstract[end-1]
        listverb.append(verb.lemma_)
dfRem["abstract"].apply(actionverb)

0       None
1       None
2       None
3       None
4       None
        ... 
2175    None
2176    None
2177    None
2178    None
2179    None
Name: abstract, Length: 2135, dtype: object

In [17]:
couverb=Counter(listverb)
couverb.most_common(20)

[('report', 79),
 ('aim', 66),
 ('find', 65),
 ('describe', 57),
 ('identify', 55),
 ('present', 51),
 ('review', 46),
 ('discuss', 44),
 ('conduct', 43),
 ('will', 40),
 ('propose', 38),
 ('use', 38),
 ('provide', 36),
 ('show', 33),
 ('perform', 33),
 ('include', 29),
 ('summarize', 28),
 ('search', 27),
 ('suggest', 20),
 ('evaluate', 19)]

# Word similarity

https://www.nltk.org/howto/wordnet.html
https://sp1819.github.io/wordnet_spacy.pdf
https://github.com/sgsinclair/alta/tree/master/ipynb
https://github.com/sgsinclair/alta/blob/master/ipynb/ArtOfLiteraryTextAnalysis.ipynb

synonyms: words with very similar meanings
hypernyms: words with more general meanings ("insect" is more general than "bug")
hyponyms: words with more specific meanings ("scarabaeus is more specific than "bug")
In 

In [28]:
from nltk.corpus import wordnet as wn
wn.synsets("review")

[Synset('reappraisal.n.01'),
 Synset('review.n.02'),
 Synset('follow-up.n.03'),
 Synset('review.n.04'),
 Synset('revue.n.01'),
 Synset('review.n.06'),
 Synset('recapitulation.n.03'),
 Synset('review.n.08'),
 Synset('review.n.09'),
 Synset('inspection.n.01'),
 Synset('review.v.01'),
 Synset('review.v.02'),
 Synset('review.v.03'),
 Synset('review.v.04'),
 Synset('review.v.05')]

In [33]:
syns=wn.synsets("conduct",pos="n")
syns

[Synset('behavior.n.01'), Synset('demeanor.n.01')]

In [35]:
for synset in wn.synsets("conduct", pos="n"):
    print(synset.name(), ": ", synset.definition(),synset.hypernyms())

behavior.n.01 :  manner of acting or controlling yourself [Synset('activity.n.01')]
demeanor.n.01 :  (behavioral attributes) the way a person behaves toward other people [Synset('trait.n.01')]


In [43]:
syn=wn.synset('review.v.01')
print(syn,syn.definition())

Synset('review.v.01') look at again; examine again


In [45]:
syn.hypernyms()

[Synset('analyze.v.01')]

# Similarity
https://towardsdatascience.com/identifying-competitors-using-word-vectorization-k-nearest-neighbors-412f1cea2506
